# UCOA to budget

2/27/2020  epq

In [ ]:
import re
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Set max_rows option for display

In [ ]:
pd.get_option("display.max_rows")

In [ ]:
pd.set_option("display.max_rows",1000)
pd.get_option("display.max_rows")
pd.set_option('display.max_columns', 50)

### Read RIDE UCOA data

Source file is: 

http://media.ride.ri.gov/construction/93-All-Expenditure-Account-Strings-with-Descriptions-no-421-422-FY18.xlsx

In [ ]:
def read_RIDE(start_fy,end_fy):
    #UCOA expense files
    path = "../RIDE_UCOA/"
    ucnames = {
        2018 : {'exp': "RIDE_UCOA_Expenditures_Excluding_Capital_and_Debt-FY18.csv",
                'cap': "RIDE_UCOA_Capital_and_Debt_Service_Expenditures-FY18.csv"},
        2017 : {'exp': "RIDE_UCOA_Expenditures_Excluding_Capital_and_Debt-FY17.csv",
                'cap': "RIDE_UCOA_Capital_and_Debt_Service_Expenditures-FY17.csv"},
        2016 : {'exp': "RIDE_UCOA_Expenditures_Excluding_Capital_and_Debt-FY16.csv",
                'cap': "RIDE_UCOA_Capital_and_Debt_Service_Expenditures-FY16.csv"},
        2015 : {'exp': "RIDE_UCOA_Expenditures_Excluding_Capital_and_Debt-FY15.csv",
                'cap': "RIDE_UCOA_Capital_and_Debt_Service_Expenditures-FY15.csv"},
        2014 : {'exp': "RIDE_UCOA_Expenditures_Excluding_Capital_and_Debt-FY14.csv",
                'cap': "RIDE_UCOA_Capital_and_Debt_Service_Expenditures-FY14.csv"},
        2013 : {'exp': "RIDE_UCOA_Expenditures_Excluding_Capital_and_Debt-FY13.csv",
                'cap': "RIDE_UCOA_Capital_and_Debt_Service_Expenditures-FY13.csv"},
        2012 : {'exp': "RIDE_UCOA_Expenditures_Excluding_Capital_and_Debt-FY12.csv",
                'cap': "RIDE_UCOA_Capital_and_Debt_Service_Expenditures-FY12.csv"},
        2011 : {'exp': "RIDE_UCOA_Expenditures_Excluding_Capital_and_Debt-FY11.csv",
                'cap': "RIDE_UCOA_Capital_and_Debt_Service_Expenditures-FY11.csv"},
        2010 : {'exp': "RIDE_UCOA_Expenditures_Excluding_Capital_and_Debt-FY10.csv",
                'cap': "RIDE_UCOA_Capital_and_Debt_Service_Expenditures-FY10.csv"}
    }
                
    
    ucd = None                                        #placeholder for dataframe
    for fy in np.arange(start_fy,end_fy+1):           #loop through fiscal years requested
        for fty in ucnames[fy].keys():
            fydf = pd.read_csv(path + ucnames[fy][fty])                   #read the file for this fiscal year
            fydf = fydf.loc[fydf['Dist No']==90.0]        #select District 90 - East Greenwich
            fydf['Year'] = fy                             #add a field for fiscal year
            if (ucd is None):                             #if this is the first fiscal year
                ucd = fydf                                #set ucd equal to the dataframe for this year
            else:                                         #otherwise concatenate the dataframe to ucd
                ucd = pd.concat([ucd,fydf],sort=False)
    return(ucd)

In [ ]:
uexp = read_RIDE(2011,2018)

In [ ]:
uexp.shape

In [ ]:
uexp.loc[(uexp['Obj']==53202.0) & (uexp['Prog'] >= 0.0)]

In [ ]:
expdd = uexp.to_dict()

In [ ]:
expdd.keys()

In [ ]:
def get_history(fyear,ucdict):
    rdict = {'fyear':fyear}
    for key in ucdict['UCOA'].keys():
        words = ucdict['UCOA'][key].split(',')
        Fund = float(words[0])
        if ('*' in words[1]):
            Loc = np.NaN
        else:
            Loc = float(words[1])
        if ('*' in words[2]):
            Func = np.NaN
        else:
            Func = float(words[2])
        Prog = float(words[3])
        if ('*' in words[4]):
            Sub = np.NaN
        else:
            Sub = float(words[4])
        Obj  = float(words[5])
    
        if 'Obj' not in rdict.keys():
            rdict['Obj'] = Obj
    
        for key in expdd['Obj'].keys():
            if (expdd['Obj'][key] == Obj):
                if ('Desc' not in rdict.keys()):
                    rdict['Desc'] = expdd['Object Description'][key]
                if (Fund == expdd['Fund'][key]):
                    if ((Func != Func) | (Func == expdd['Func'][key])):
                        if (expdd['Prog'][key] == Prog):
                            if ((Sub != Sub) | (Sub == expdd['Sub'][key])):
                                if ((Loc != Loc) | (Loc == expdd['Loc'][key])):
                                    if (fyear == expdd['Year'][key]):
                                        if ('Actual' not in rdict.keys()):
                                            rdict['Actual'] = expdd['Actual'][key]
                                        else:
                                            rdict['Actual'] += expdd['Actual'][key]
                                        if ('Budget' not in rdict.keys()):
                                            rdict['Budget'] = expdd['Budget'][key]
                                        else:
                                            rdict['Budget'] += expdd['Budget'][key]
                                
                            
                            
    return(rdict);

In [ ]:
def get_ucoa_codes(b,budg,bsect,line):
    try:
        ucs = (b[budg][bsect][line]['UCOA'][1])
        words1 = ucs.split(',')
        uc_string = ''
        for word in words1:
            words1 = word.split('.')
            if (len(words1[0]) > 7):
                words1[0] = words1[0][:4]
            uc_string = uc_string + words1[0] + '-' 
        uc_string = uc_string[:len(uc_string)-3]
        return(uc_string)
    except IndexError:
        print('no match for: ',budget,bsect,sect)
        return('          ')

In [ ]:
for ix in expdd['Obj'].keys():
    if (expdd['Obj'][ix]==54202.0):
        print(expdd['Obj'][ix],expdd['Object Description'][ix],expdd['Loc'][ix],expdd['Year'][ix], \
              expdd['Actual'][ix],expdd['Budget'][ix])

In [ ]:
df = uexp.loc[(uexp['Obj']==55111.0) & (uexp['Prog'] == 10.0) & (uexp['Year'] == 2018) & (uexp['Loc'] == 5106.0)]
df.groupby(['Sub']).sum()
#df

In [ ]:
#Budget sections:
#Purchased Professional & Technical Services
#Purchased Property Services
#Other Purchased Services
#Supplies
#Property
#Miscellaneous Expenses

budget_sections = {1:'Purchased Professional & Technical Services',
            2:'Purchased Property Services',
            3:'Other Purchased Services',
            4:'Supplies',
            5:'Property',
            6:'Miscellaneous Expenses'}

In [ ]:

budget = {}

budget['EGHS Princ/Main Office'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Princ/Main Office'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,0.0   ,53301.0,0.0'}}
budget['EGHS Princ/Main Office'][1][2] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,0.0000,53303.0,0.0'}}
budget['EGHS Princ/Main Office'][1][3] = {'UCOA': {1: '10000000.0,5106.0,511.0,10.0,0.0000,53303.0,0.0'}}
budget['EGHS Princ/Main Office'][1][4] = {'UCOA': {1: '10000000.0,5106.0,512.0,10.0,*     ,53406.0,0.0'}}
budget['EGHS Princ/Main Office'][1][5] = {'UCOA': {1: '10000000.0,5106.0,214.0,10.0,*     ,53502.0,0.0'}}
budget['EGHS Princ/Main Office'][1][6] = {'UCOA': {1: '10000000.0,5106.0,512.0,10.0,*     ,53705.0,0.0'}}
#Other Purchased Services
budget['EGHS Princ/Main Office'][3][1] = {'UCOA': {1: '10000000.0,5106.0,512.0,10.0,*     ,55501.0,0.0'}}
budget['EGHS Princ/Main Office'][3][2] = {'UCOA': {1: '10000000.0,5106.0,512.0,10.0,*     ,55503.0,0.0'}}
#Supplies
budget['EGHS Princ/Main Office'][4][1] = {'UCOA': {1: '10000000.0,5106.0,511.0,10.0,0.0   ,56101.0,0.0'}}
budget['EGHS Princ/Main Office'][4][2] = {'UCOA': {1: '10000000.0,5106.0,512.0,10.0,0.0   ,56101.0,0.0'}}
budget['EGHS Princ/Main Office'][4][3] = {'UCOA': {1: '10000000.0,5106.0,512.0,10.0,*     ,56112.0,0.0'}}
budget['EGHS Princ/Main Office'][4][4] = {'UCOA': {1: '10000000.0,5106.0,214.0,10.0,*     ,56113.0,0.0'}}
#Property
budget['EGHS Princ/Main Office'][5][1] = {'UCOA': {1: '10000000.0,5106.0,512.0,10.0,*     ,57305.0,0.0'}}
budget['EGHS Princ/Main Office'][5][2] = {'UCOA': {1: '10000000.0,5106.0,512.0,10.0,*     ,57306.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Princ/Main Office'][6][1] = {'UCOA': {1: '10000000.0,5106.0,511.0,10.0,0.0   ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS Art'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Art'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,200.0 ,53301.0,0.0'}}
#Purchased Property Services
budget['EGHS Art'][2][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,200.0 ,54311.0,0.0'}}
#Supplies
budget['EGHS Art'][4][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,200.0 ,56101.0,0.0'}}
#Property
budget['EGHS Art'][5][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,200.0 ,57305.0,0.0'}}
budget['EGHS Art'][5][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,200.0 ,57306.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Art'][6][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,200.0 ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS English Language Arts'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS English Language Arts'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,500.0 ,53301.0,0.0'}}
budget['EGHS English Language Arts'][1][2] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,500.0 ,53303.0,0.0'}}
#Other Purchased Services
budget['EGHS English Language Arts'][3][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,500.0 ,55111.0,0.0'}}
budget['EGHS English Language Arts'][3][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,500.0 ,55809.0,0.0'}}
budget['EGHS English Language Arts'][3][3] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,500.0 ,55809.0,0.0'}}
#Supplies
budget['EGHS English Language Arts'][4][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,500.0 ,56101.0,0.0'}}
budget['EGHS English Language Arts'][4][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,500.0 ,56401.0,0.0'}}
budget['EGHS English Language Arts'][4][3] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,500.0 ,56501.0,0.0'}}
#Property
budget['EGHS English Language Arts'][5][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,500.0 ,57305.0,0.0'}}
budget['EGHS English Language Arts'][5][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,500.0 ,57306.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS English Language Arts'][6][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,500.0 ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS Foreign Languages'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Foreign Languages'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,700.0 ,53301.0,0.0'}}
#Other Purchased Services
budget['EGHS Foreign Languages'][3][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,700.0 ,55111.0,0.0'}}
budget['EGHS Foreign Languages'][3][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,700.0 ,55809.0,0.0'}}
budget['EGHS Foreign Languages'][3][3] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,700.0 ,55809.0,0.0'}}
#Supplies
budget['EGHS Foreign Languages'][4][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,700.0 ,56101.0,0.0'}}
budget['EGHS Foreign Languages'][4][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,700.0 ,56401.0,0.0'}}
budget['EGHS Foreign Languages'][4][3] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,700.0 ,56404.0,0.0'}}
#Property
budget['EGHS Foreign Languages'][5][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,700.0 ,57305.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Foreign Languages'][6][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,700.0 ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS Guidance'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Guidance'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,800.0 ,53303.0,0.0'}}
budget['EGHS Guidance'][1][2] = {'UCOA': {1: '10000000.0,5106.0,211.0,10.0,800.0 ,53503.0,0.0'}}
budget['EGHS Guidance'][1][3] = {'UCOA': {1: '10000000.0,5106.0,211.0,10.0,800.0 ,53705.0,0.0'}}
#Other Purchased Services
budget['EGHS Guidance'][3][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,800.0 ,55809.0,0.0'}}
#Supplies
budget['EGHS Guidance'][4][1] = {'UCOA': {1: '10000000.0,5106.0,211.0,10.0,800.0 ,56101.0,0.0'}}
budget['EGHS Guidance'][4][2] = {'UCOA': {1: '10000000.0,5106.0,211.0,10.0,800.0 ,56403.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Guidance'][6][1] = {'UCOA': {1: '10000000.0,5106.0,211.0,10.0,800.0 ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS Physical Education and Health'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Physical Education and Health'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1200.0 ,53301.0,0.0'}}
budget['EGHS Physical Education and Health'][1][2] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1200.0 ,53303.0,0.0'}}
#Other Purchased Services
budget['EGHS Physical Education and Health'][3][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1200.0 ,55809.0,0.0'}}
#Supplies
budget['EGHS Physical Education and Health'][4][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1200.0 ,56101.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Physical Education and Health'][6][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1200.0 ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS Business Education'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Business Education'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1413.0 ,53301.0,0.0'}}
#Other Purchased Services
budget['EGHS Business Education'][3][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1800.0 ,55111.0,0.0'}}
budget['EGHS Business Education'][3][2] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1413.0 ,55809.0,0.0'}}
#Supplies
budget['EGHS Business Education'][4][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1800.0 ,56101.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Business Education'][6][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1800.0 ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS Mathematics'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Mathematics'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1500.0 ,53301.0,0.0'}}
budget['EGHS Mathematics'][1][2] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1500.0 ,53303.0,0.0'}}
#Other Purchased Services
budget['EGHS Mathematics'][3][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1500.0 ,55111.0,0.0'}}
budget['EGHS Mathematics'][3][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1500.0 ,55809.0,0.0'}}
budget['EGHS Mathematics'][3][3] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1500.0 ,55809.0,0.0'}}
#Supplies
budget['EGHS Mathematics'][4][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1500.0 ,56101.0,0.0'}}
budget['EGHS Mathematics'][4][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1500.0 ,56401.0,0.0'}}
#Property
budget['EGHS Mathematics'][5][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1500.0 ,57305.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Mathematics'][6][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1500.0 ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS Music/Theater and Performing'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Music/Theater and Performing'][1][1] = {'UCOA': {1: '10000000.0,5106.0,111.0,10.0,1600.0 ,53210.0,0.0'}}
#Purchased Property Services
budget['EGHS Music/Theater and Performing'][2][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1600.0 ,54311.0,0.0'}}
#Other Purchased Services
budget['EGHS Music/Theater and Performing'][3][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1600.0 ,55111.0,0.0'}}
#Supplies
budget['EGHS Music/Theater and Performing'][4][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1600.0 ,56101.0,0.0'}}
#Property
budget['EGHS Music/Theater and Performing'][5][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1600.0 ,57305.0,0.0'}}
budget['EGHS Music/Theater and Performing'][5][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1600.0 ,57306.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Music/Theater and Performing'][6][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1600.0 ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS Natural Sciences'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Natural Sciences'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1700.0 ,53301.0,0.0'}}
#Other Purchased Services
budget['EGHS Natural Sciences'][3][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1700.0 ,55111.0,0.0'}}
budget['EGHS Natural Sciences'][3][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1700.0 ,55809.0,0.0'}}
budget['EGHS Natural Sciences'][3][3] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1700.0 ,55809.0,0.0'}}
#Supplies
budget['EGHS Natural Sciences'][4][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1700.0 ,56101.0,0.0'}}
budget['EGHS Natural Sciences'][4][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1700.0 ,56401.0,0.0'}}
budget['EGHS Natural Sciences'][4][3] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1700.0 ,56404.0,0.0'}}
#Property
budget['EGHS Natural Sciences'][5][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1700.0 ,57305.0,0.0'}}
budget['EGHS Natural Sciences'][5][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1700.0 ,57306.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Natural Sciences'][6][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1700.0 ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS Social Sciences'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Natural Sciences'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1900.0 ,53301.0,0.0'}}
budget['EGHS Natural Sciences'][1][2] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1900.0 ,53303.0,0.0'}}
#Other Purchased Services
budget['EGHS Social Sciences'][3][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1900.0 ,55111.0,0.0'}}
budget['EGHS Social Sciences'][3][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1900.0 ,55809.0,0.0'}}
budget['EGHS Social Sciences'][3][3] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,1900.0 ,55809.0,0.0'}}
#Supplies
budget['EGHS Social Sciences'][4][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1900.0 ,56101.0,0.0'}}
budget['EGHS Social Sciences'][4][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1900.0 ,56401.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Social Sciences'][6][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,1900.0 ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS Technical Education/Computer'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Technical Education/Computer'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,2000.0 ,53301.0,0.0'}}
#Purchased Property Services
budget['EGHS Technical Education/Computer'][2][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,2000.0 ,54311.0,0.0'}}
#Other Purchased Services
budget['EGHS Technical Education/Computer'][3][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,2000.0 ,55111.0,0.0'}}
#Supplies
budget['EGHS Technical Education/Computer'][4][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,2000.0 ,56101.0,0.0'}}
budget['EGHS Technical Education/Computer'][4][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,2000.0 ,56112.0,0.0'}}
#Property
budget['EGHS Technical Education/Computer'][5][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,2000.0 ,57305.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Technical Education/Computer'][6][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,2000.0 ,58102.0,0.0'}}
###########################################################################################################
budget['EGHS Literacy and Reading'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Literacy and Reading'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,2400.0 ,53301.0,0.0'}}
#Supplies
budget['EGHS Literacy and Reading'][4][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,2400.0 ,56101.0,0.0'}}
budget['EGHS Literacy and Reading'][4][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,2400.0 ,56401.0,0.0'}}
budget['EGHS Literacy and Reading'][4][2] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,2400.0 ,56404.0,0.0'}}
#Property
budget['EGHS Literacy and Reading'][5][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,2400.0 ,57305.0,0.0'}}
budget['EGHS Literacy and Reading'][5][1] = {'UCOA': {1: '10000000.0,5106.0,122.0,10.0,2400.0 ,57306.0,0.0'}}
###########################################################################################################
budget['EGHS Nursing'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Nursing'][1][1] = {'UCOA': {1: '10000000.0,5106.0,511.0,10.0,2500.0 ,53301.0,0.0'}}
budget['EGHS Nursing'][1][2] = {'UCOA': {1: '10000000.0,5106.0,216.0,10.0,2500.0 ,53411.0,0.0'}}
budget['EGHS Nursing'][1][2] = {'UCOA': {1: '10000000.0,5106.0,216.0,10.0,2500.0 ,53412.0,0.0'}}
budget['EGHS Nursing'][1][2] = {'UCOA': {1: '10000000.0,5106.0,216.0,10.0,2500.0 ,53417.0,0.0'}}
#Supplies
budget['EGHS Nursing'][4][1] = {'UCOA': {1: '10000000.0,5106.0,216.0,10.0,2500.0 ,56115.0,0.0'}}
#Property
budget['EGHS Nursing'][5][1] = {'UCOA': {1: '10000000.0,5106.0,216.0,10.0,2500.0 ,57305.0,0.0'}}
###########################################################################################################
budget['EGHS Library Science'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['EGHS Library Science'][1][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,2600.0 ,53301.0,0.0'}}
budget['EGHS Library Science'][1][2] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,2600.0 ,53303.0,0.0'}}
#Other Purchased Services
budget['EGHS Library Science'][3][1] = {'UCOA': {1: '10000000.0,5106.0,222.0,10.0,2600.0 ,55809.0,0.0'}}
#Supplies
budget['EGHS Library Science'][4][1] = {'UCOA': {1: '10000000.0,5106.0,212.0,10.0,2600.0 ,56101.0,0.0'}}
budget['EGHS Library Science'][4][2] = {'UCOA': {1: '10000000.0,5106.0,212.0,10.0,2600.0 ,56401.0,0.0'}}
budget['EGHS Library Science'][4][2] = {'UCOA': {1: '10000000.0,5106.0,212.0,10.0,2600.0 ,56401.0,0.0'}}
#Miscellaneous Expenses
budget['EGHS Library Science'][6][1] = {'UCOA': {1: '10000000.0,5106.0,212.0,10.0,2600.0 ,58102.0,0.0'}}
###########################################################################################################
budget['Archie R Cole Middle School'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['Archie R Cole Middle School'][1][1] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,53210.0,0.0'}}
budget['Archie R Cole Middle School'][1][2] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,53216.0,0.0'}}
budget['Archie R Cole Middle School'][1][3] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,53220.0,0.0'}}
budget['Archie R Cole Middle School'][1][4] = {'UCOA': {1: '10000000.0,4103.0,222.0,10.0,*     ,53301.0,0.0'}}
budget['Archie R Cole Middle School'][1][5] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,53303.0,0.0'}}
budget['Archie R Cole Middle School'][1][6] = {'UCOA': {1: '10000000.0,4103.0,512.0,10.0,*     ,53406.0,0.0'}}
budget['Archie R Cole Middle School'][1][7] = {'UCOA': {1: '10000000.0,4103.0,214.0,10.0,*     ,53502.0,0.0'}}
budget['Archie R Cole Middle School'][1][8] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,53503.0,0.0'}}
budget['Archie R Cole Middle School'][1][9] = {'UCOA': {1: '10000000.0,4103.0,512.0,10.0,*     ,53705.0,0.0'}}
budget['Archie R Cole Middle School'][1][10]= {'UCOA': {1: '10000000.0,4103.0,216.0,10.0,2500.0,53417.0,0.0'}}
#Other Purchased Services
budget['Archie R Cole Middle School'][3][1] = {'UCOA': {1: '10000000.0,4103.0,512.0,10.0,1200.0,55111.0,0.0'}}
budget['Archie R Cole Middle School'][3][1] = {'UCOA': {1: '10000000.0,4103.0,512.0,10.0,*     ,55501.0,0.0'}}
budget['Archie R Cole Middle School'][3][2] = {'UCOA': {1: '10000000.0,4103.0,512.0,10.0,*     ,55503.0,0.0'}}
budget['Archie R Cole Middle School'][3][3] = {'UCOA': {1: '10000000.0,4103.0,511.0,10.0,*     ,55803.0,0.0'}}
budget['Archie R Cole Middle School'][3][4] = {'UCOA': {1: '10000000.0,4103.0,222.0,10.0,*     ,55809.0,0.0'}}
#Supplies
budget['Archie R Cole Middle School'][4][1] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,56101.0,0.0'}}
budget['Archie R Cole Middle School'][4][2] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,56113.0,0.0'}}
budget['Archie R Cole Middle School'][4][3] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,56115.0,0.0'}}
budget['Archie R Cole Middle School'][4][4] = {'UCOA': {1: '10000000.0,4103.0,122.0,10.0,*     ,56401.0,0.0'}}
budget['Archie R Cole Middle School'][4][5] = {'UCOA': {1: '10000000.0,4103.0,212.0,10.0,*     ,56402.0,0.0'}}
budget['Archie R Cole Middle School'][4][6] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,56403.0,0.0'}}
budget['Archie R Cole Middle School'][4][7] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,56404.0,0.0'}}

#Property
budget['Archie R Cole Middle School'][5][1] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,57305.0,0.0'}}
budget['Archie R Cole Middle School'][5][2] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,57306.0,0.0'}}
#Miscellaneous Expenses
budget['Archie R Cole Middle School'][6][1] = {'UCOA': {1: '10000000.0,4103.0,*    ,10.0,*     ,58102.0,0.0'}}
##########################################################################################################
budget['James H Eldredge School'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['James H Eldredge School'][1][1] = {'UCOA': {1: '10000000.0,3102.0,222.0,10.0,*     ,53301.0,0.0'}}
budget['James H Eldredge School'][1][2] = {'UCOA': {1: '10000000.0,3102.0,*    ,10.0,*     ,53303.0,0.0'}}
budget['James H Eldredge School'][1][3] = {'UCOA': {1: '10000000.0,3102.0,214.0,10.0,*     ,53502.0,0.0'}}
budget['James H Eldredge School'][1][4] = {'UCOA': {1: '10000000.0,3102.0,*    ,10.0,*     ,53503.0,0.0'}}
budget['James H Eldredge School'][1][5] = {'UCOA': {1: '10000000.0,3102.0,512.0,10.0,*     ,53705.0,0.0'}}
budget['James H Eldredge School'][1][6] = {'UCOA': {1: '10000000.0,3102.0,216.0,10.0,2500.0,53417.0,0.0'}}
#Other Purchased Services
budget['James H Eldredge School'][3][1] = {'UCOA': {1: '10000000.0,3102.0,512.0,10.0,*     ,55501.0,0.0'}}
budget['James H Eldredge School'][3][2] = {'UCOA': {1: '10000000.0,3102.0,512.0,10.0,*     ,55503.0,0.0'}}
budget['James H Eldredge School'][3][3] = {'UCOA': {1: '10000000.0,3102.0,511.0,10.0,*     ,55803.0,0.0'}}
budget['James H Eldredge School'][3][4] = {'UCOA': {1: '10000000.0,3102.0,222.0,10.0,*     ,55809.0,0.0'}}
#Supplies
budget['James H Eldredge School'][4][1] = {'UCOA': {1: '10000000.0,3102.0,*    ,10.0,*     ,56101.0,0.0'}}
budget['James H Eldredge School'][4][2] = {'UCOA': {1: '10000000.0,3102.0,*    ,10.0,2500.0,56115.0,0.0'}}
budget['James H Eldredge School'][4][3] = {'UCOA': {1: '10000000.0,3102.0,122.0,10.0,*     ,56401.0,0.0'}}
budget['James H Eldredge School'][4][4] = {'UCOA': {1: '10000000.0,3102.0,212.0,10.0,*     ,56402.0,0.0'}}
budget['James H Eldredge School'][4][5] = {'UCOA': {1: '10000000.0,3102.0,*    ,10.0,*     ,56404.0,0.0'}}
#Property
budget['James H Eldredge School'][5][1] = {'UCOA': {1: '10000000.0,3102.0,*    ,10.0,*     ,57305.0,0.0'}}
budget['James H Eldredge School'][5][2] = {'UCOA': {1: '10000000.0,3102.0,*    ,10.0,*     ,57306.0,0.0'}}
#Miscellaneous Expenses
budget['James H Eldredge School'][6][1] = {'UCOA': {1: '10000000.0,3102.0,*    ,10.0,*     ,58102.0,0.0'}}
##########################################################################################################
budget['George Hanaford School'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['George Hanaford School'][1][1] = {'UCOA': {1: '10000000.0,3108.0,214.0,20.0,2101.0,53207.0,0.0'}}
budget['George Hanaford School'][1][2] = {'UCOA': {1: '10000000.0,3108.0,222.0,10.0,*     ,53301.0,0.0'}}
budget['George Hanaford School'][1][3] = {'UCOA': {1: '10000000.0,3108.0,222.0,10.0,*     ,53303.0,0.0'}}
budget['George Hanaford School'][1][4] = {'UCOA': {1: '10000000.0,3108.0,214.0,10.0,*     ,53502.0,0.0'}}
budget['George Hanaford School'][1][5] = {'UCOA': {1: '10000000.0,3108.0,*    ,10.0,*     ,53503.0,0.0'}}
budget['George Hanaford School'][1][6] = {'UCOA': {1: '10000000.0,3108.0,512.0,10.0,*     ,53705.0,0.0'}}
budget['George Hanaford School'][1][7] = {'UCOA': {1: '10000000.0,3108.0,216.0,10.0,2500.0,53417.0,0.0'}}
#Other Purchased Services
budget['George Hanaford School'][3][1] = {'UCOA': {1: '10000000.0,3108.0,512.0,10.0,*     ,55501.0,0.0'}}
budget['George Hanaford School'][3][2] = {'UCOA': {1: '10000000.0,3108.0,512.0,10.0,*     ,55503.0,0.0'}}
budget['George Hanaford School'][3][3] = {'UCOA': {1: '10000000.0,3108.0,511.0,10.0,*     ,55803.0,0.0'}}
budget['George Hanaford School'][3][4] = {'UCOA': {1: '10000000.0,3108.0,222.0,10.0,*     ,55809.0,0.0'}}
#Supplies
budget['George Hanaford School'][4][1] = {'UCOA': {1: '10000000.0,3108.0,*    ,10.0,*     ,56101.0,0.0'}}
budget['George Hanaford School'][4][2] = {'UCOA': {1: '10000000.0,3108.0,*    ,10.0,2500.0,56115.0,0.0'}}
budget['George Hanaford School'][4][3] = {'UCOA': {1: '10000000.0,3108.0,122.0,10.0,*     ,56401.0,0.0'}}
budget['George Hanaford School'][4][4] = {'UCOA': {1: '10000000.0,3108.0,212.0,10.0,*     ,56402.0,0.0'}}
budget['George Hanaford School'][4][5] = {'UCOA': {1: '10000000.0,3108.0,*    ,10.0,*     ,56404.0,0.0'}}
#Property
budget['George Hanaford School'][5][1] = {'UCOA': {1: '10000000.0,3108.0,*    ,10.0,*     ,57305.0,0.0'}}
budget['George Hanaford School'][5][2] = {'UCOA': {1: '10000000.0,3108.0,*    ,10.0,*     ,57306.0,0.0'}}
#Miscellaneous Expenses
budget['George Hanaford School'][6][1] = {'UCOA': {1: '10000000.0,3108.0,*    ,10.0,*     ,58102.0,0.0'}}
#########################################################################################################
budget['Frenchtown School'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['Frenchtown School'][1][1] = {'UCOA': {1: '10000000.0,3105.0,111.0,10.0,1600.0,53210.0,0.0'}}
budget['Frenchtown School'][1][2] = {'UCOA': {1: '10000000.0,3105.0,222.0,10.0,*     ,53301.0,0.0'}}
budget['Frenchtown School'][1][3] = {'UCOA': {1: '10000000.0,3105.0,*    ,10.0,*     ,53303.0,0.0'}}
budget['Frenchtown School'][1][4] = {'UCOA': {1: '10000000.0,3105.0,214.0,10.0,*     ,53502.0,0.0'}}
budget['Frenchtown School'][1][5] = {'UCOA': {1: '10000000.0,3105.0,*    ,10.0,*     ,53503.0,0.0'}}
budget['Frenchtown School'][1][6] = {'UCOA': {1: '10000000.0,3105.0,512.0,10.0,*     ,53705.0,0.0'}}
budget['Frenchtown School'][1][7] = {'UCOA': {1: '10000000.0,3105.0,216.0,10.0,2500.0,53417.0,0.0'}}
#Other Purchased Services
budget['Frenchtown School'][3][1] = {'UCOA': {1: '10000000.0,3105.0,512.0,10.0,*     ,55501.0,0.0'}}
budget['Frenchtown School'][3][2] = {'UCOA': {1: '10000000.0,3105.0,512.0,10.0,*     ,55503.0,0.0'}}
budget['Frenchtown School'][3][3] = {'UCOA': {1: '10000000.0,3105.0,511.0,10.0,*     ,55803.0,0.0'}}
budget['Frenchtown School'][3][4] = {'UCOA': {1: '10000000.0,3105.0,222.0,10.0,*     ,55809.0,0.0'}}
#Supplies
budget['Frenchtown School'][4][1] = {'UCOA': {1: '10000000.0,3105.0,*    ,10.0,*     ,56101.0,0.0'}}
budget['Frenchtown School'][4][2] = {'UCOA': {1: '10000000.0,3105.0,*    ,10.0,2500.0,56115.0,0.0'}}
budget['Frenchtown School'][4][3] = {'UCOA': {1: '10000000.0,3105.0,122.0,10.0,*     ,56401.0,0.0'}}
budget['Frenchtown School'][4][4] = {'UCOA': {1: '10000000.0,3105.0,212.0,10.0,*     ,56402.0,0.0'}}
budget['Frenchtown School'][4][5] = {'UCOA': {1: '10000000.0,3105.0,*    ,10.0,*     ,56404.0,0.0'}}
#Property
budget['Frenchtown School'][5][1] = {'UCOA': {1: '10000000.0,3105.0,*    ,10.0,*     ,57305.0,0.0'}}
budget['Frenchtown School'][5][2] = {'UCOA': {1: '10000000.0,3105.0,*    ,10.0,*     ,57306.0,0.0'}}
#Miscellaneous Expenses
budget['Frenchtown School'][6][1] = {'UCOA': {1: '10000000.0,3105.0,*    ,10.0,*     ,58102.0,0.0'}}
###########################################################################################################
budget['Meadowbrook School'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['Meadowbrook School'][1][1] = {'UCOA': {1: '10000000.0,3107.0,214.0,20.0,2101.0,53207.0,0.0'}}
budget['Meadowbrook School'][1][2] = {'UCOA': {1: '10000000.0,3107.0,222.0,10.0,*     ,53301.0,0.0'}}
budget['Meadowbrook School'][1][3] = {'UCOA': {1: '10000000.0,3107.0,222.0,10.0,*     ,53303.0,0.0'}}
budget['Meadowbrook School'][1][4] = {'UCOA': {1: '10000000.0,3107.0,214.0,10.0,*     ,53502.0,0.0'}}
budget['Meadowbrook School'][1][5] = {'UCOA': {1: '10000000.0,3107.0,*    ,10.0,*     ,53503.0,0.0'}}
budget['Meadowbrook School'][1][6] = {'UCOA': {1: '10000000.0,3107.0,512.0,10.0,*     ,53705.0,0.0'}}
budget['Meadowbrook School'][1][7] = {'UCOA': {1: '10000000.0,3107.0,216.0,10.0,2500.0,53417.0,0.0'}}
#Other Purchased Services
budget['Meadowbrook School'][3][1] = {'UCOA': {1: '10000000.0,3107.0,512.0,10.0,*     ,55501.0,0.0'}}
budget['Meadowbrook School'][3][2] = {'UCOA': {1: '10000000.0,3107.0,512.0,10.0,*     ,55503.0,0.0'}}
budget['Meadowbrook School'][3][3] = {'UCOA': {1: '10000000.0,3107.0,511.0,10.0,*     ,55803.0,0.0'}}
budget['Meadowbrook School'][3][4] = {'UCOA': {1: '10000000.0,3107.0,222.0,10.0,*     ,55809.0,0.0'}}
#Supplies
budget['Meadowbrook School'][4][1] = {'UCOA': {1: '10000000.0,3107.0,*    ,10.0,*     ,56101.0,0.0'}}
budget['Meadowbrook School'][4][2] = {'UCOA': {1: '10000000.0,3107.0,*    ,10.0,2500.0,56115.0,0.0'}}
budget['Meadowbrook School'][4][3] = {'UCOA': {1: '10000000.0,3107.0,122.0,10.0,*     ,56401.0,0.0'}}
budget['Meadowbrook School'][4][4] = {'UCOA': {1: '10000000.0,3107.0,212.0,10.0,*     ,56402.0,0.0'}}
budget['Meadowbrook School'][4][5] = {'UCOA': {1: '10000000.0,3107.0,*    ,10.0,*     ,56404.0,0.0'}}
#Property
budget['Meadowbrook School'][5][1] = {'UCOA': {1: '10000000.0,3107.0,*    ,10.0,*     ,57305.0,0.0'}}
budget['Meadowbrook School'][5][2] = {'UCOA': {1: '10000000.0,3107.0,*    ,10.0,*     ,57306.0,0.0'}}
#Miscellaneous Expenses
budget['Meadowbrook School'][6][1] = {'UCOA': {1: '10000000.0,3107.0,*    ,10.0,*     ,58102.0,0.0'}}
##########################################################################################################
budget['Teaching and Learning'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['Teaching and Learning'][1][1] = {'UCOA': {1: '10000000.0,*     ,*     ,10.0,0.0   ,53214.0,0.0'}}
budget['Teaching and Learning'][1][2] = {'UCOA': {1: '10000000.0,*     ,*     ,10.0,*     ,53301.0,0.0',
                                                  2: '10000000.0,*     ,*     ,20.0,*     ,53301.0,0.0'}}
budget['Teaching and Learning'][1][3] = {'UCOA': {1: '10000000.0,*     ,*     ,10.0,*     ,52917.0,0.0',
                                                  2: '10000000.0,*     ,*     ,20.0,*     ,52917.0,0.0'}}
##########################################################################################################
budget['Student Services'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['Student Services'][1][1] = {'UCOA': {1: '10000000.0,*     ,232.0,20.0,2122.0,53202.0,0.0'}}
budget['Student Services'][1][2] = {'UCOA': {1: '10000000.0,*     ,232.0,20.0,2125.0,53203.0,0.0'}}
budget['Student Services'][1][3] = {'UCOA': {1: '10000000.0,*     ,232.0,20.0,2121.0,53205.0,0.0'}}
budget['Student Services'][1][4] = {'UCOA': {1: '10000000.0,1400.0,214.0,20.0,2101.0,53207.0,0.0'}}
budget['Student Services'][1][5] = {'UCOA': {1: '10000000.0,*     ,232.0,20.0,2119.0,53208.0,0.0'}}
budget['Student Services'][1][6] = {'UCOA': {1: '10000000.0,*     ,232.0,20.0,2124.0,53211.0,0.0'}}
budget['Student Services'][1][7] = {'UCOA': {1: '10000000.0,*     ,232.0,20.0,2121.0,53213.0,0.0'}}
budget['Student Services'][1][8] = {'UCOA': {1: '10000000.0,*     ,232.0,20.0,2103.0,53220.0,0.0'}}
budget['Student Services'][1][9] = {'UCOA': {1: '10000000.0,*     ,222.0,20.0,*     ,53301.0,0.0'}}
budget['Student Services'][1][10]= {'UCOA': {1: '10000000.0,*     ,222.0,20.0,*     ,53303.0,0.0',
                                             2: '10000000.0,*     ,231.0,20.0,2130.0,53303.0,0.0'}}
budget['Student Services'][1][11]= {'UCOA': {1: '10000000.0,*     ,532.0,20.0,2130.0,53402.0,0.0'}}
budget['Student Services'][1][12]= {'UCOA': {1: '10000000.0,*     ,231.0,20.0,2131.0,53414.0,0.0'}}
budget['Student Services'][1][13]= {'UCOA': {1: '10000000.0,*     ,216.0,20.0,2131.0,53417.0,0.0'}}
budget['Student Services'][1][14]= {'UCOA': {1: '10000000.0,*     ,122.0,20.0,*     ,53503.0,0.0',
                                             2: '10000000.0,*     ,122.0,40.0,600.0 ,53503.0,0.0',
                                             3: '10000000.0,*     ,232.0,20.0,*     ,53503.0,0.0'}}
budget['Student Services'][1][15]= {'UCOA': {1: '10000000.0,*     ,231.0,20.0,*     ,53705.0,0.0'}}
#Purchased Property Services
#Other Purchased Services
budget['Student Services'][3][1] = {'UCOA': {1: '10000000.0,*     ,431.0,20.0,*     ,55630.0,0.0'}}
budget['Student Services'][3][2] = {'UCOA': {1: '10000000.0,*     ,431.0,20.0,*     ,55640.0,0.0'}}
budget['Student Services'][3][3] = {'UCOA': {1: '10000000.0,*     ,232.0,20.0,2500.0,55803.0,0.0'}}
budget['Student Services'][3][4] = {'UCOA': {1: '10000000.0,*     ,122.0,40.0,600.0 ,55809.0,0.0',                                             2: '10000000.0,*     ,222.0,20.0,2101.0,55809.0,0.0'}}
#Supplies
budget['Student Services'][4][1] = {'UCOA': {1: '10000000.0,*     ,232.0,20.0,*     ,56101.0,0.0',
                                             2: '10000000.0,*     ,122.0,20.0,*     ,56101.0,0.0',
                                             3: '10000000.0,*     ,122.0,40.0,600.0 ,56101.0,0.0'}}
budget['Student Services'][4][2] = {'UCOA': {1: '10000000.0,*     ,122.0,20.0,2103.0,56401.0,0.0'}}
budget['Student Services'][4][3] = {'UCOA': {1: '10000000.0,*     ,232.0,20.0,2103.0,56404.0,0.0'}}
budget['Student Services'][4][3] = {'UCOA': {1: '10000000.0,*     ,*    ,20.0,*     ,56501.0,0.0'}}
#Property
budget['Student Services'][5][1] = {'UCOA': {1: '10000000.0,*     ,122.0,20.0,*     ,57305.0,0.0',
                                             2: '10000000.0,*     ,232.0,20.0,*     ,57305.0,0.0'}}
budget['Student Services'][5][2] = {'UCOA': {1: '10000000.0,*     ,122.0,20.0,*     ,57306.0,0.0'}}
#Miscellaneous Expenses
budget['Student Services'][6][1] = {'UCOA': {1: '10000000.0,*     ,*    ,20.0,*     ,58101.0,0.0'}}
budget['Student Services'][6][2] = {'UCOA': {1: '10000000.0,*     ,231.0,20.0,2130.0,58102.0,0.0'}}
#######################################################################################################
budget['Athletics'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['Athletics'][1][1] = {'UCOA': {1: '10000000.0,4103.0,213.0,90.0,2200.0,53303.0,0.0'}}
budget['Athletics'][1][2] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,53410.0,0.0'}}
budget['Athletics'][1][3] = {'UCOA': {1: '10000000.0,4103.0,213.0,90.0,2200.0,53416.0,0.0'}}
budget['Athletics'][1][4] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,53416.0,0.0'}}
budget['Athletics'][1][4] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,53417.0,0.0'}}
#Purchased Property Services
budget['Athletics'][2][1] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,54206.0,0.0'}}
budget['Athletics'][2][2] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,54601.0,0.0'}}
budget['Athletics'][2][3] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,54605.0,0.0'}}
budget['Athletics'][2][4] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,54606.0,0.0'}}
#Other Purchased Services
budget['Athletics'][3][1] = {'UCOA': {1: '10000000.0,*     ,213.0,90.0,2200.0,55111.0,0.0'}}
budget['Athletics'][3][2] = {'UCOA': {1: '10000000.0,*     ,213.0,90.0,2200.0,55810.0,0.0'}}
#Supplies
budget['Athletics'][4][1] = {'UCOA': {1: '10000000.0,4103.0,213.0,90.0,2200.0,56101.0,0.0',
                                      2: '10000000.0,5106.0,213.0,90.0,2200.0,56101.0,0.0'}}
budget['Athletics'][4][2] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,56112.0,0.0'}}
budget['Athletics'][4][3] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,56115.0,0.0'}}
budget['Athletics'][4][4] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,56116.0,0.0'}}
budget['Athletics'][4][5] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,56117.0,0.0'}}
#Property
budget['Athletics'][5][1] = {'UCOA': {1: '10000000.0,4103.0,213.0,90.0,2200.0,57305.0,0.0'}}
budget['Athletics'][5][2] = {'UCOA': {1: '10000000.0,5106.0,213.0,90.0,2200.0,57305.0,0.0'}}
#Miscellaneous Expenses
budget['Athletics'][6][1] = {'UCOA': {1: '10000000.0,*     ,213.0,90.0,2200.0,58102.0,0.0'}}
#####################################################################################################
budget['Facilities'] = {1:{},2:{},3:{},4:{},5:{},6:{}}
#Purchased Professional & Technical Services
budget['Facilities'][1][1] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,53301.0,0.0'}}
budget['Facilities'][1][2] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,53303.0,0.0'}}
budget['Facilities'][1][3] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,53406.0,0.0'}}
budget['Facilities'][1][4] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,53502.0,0.0'}}
#Purchased Property Services
budget['Facilities'][2][1] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,54201.0,0.0'}}
budget['Facilities'][2][2] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,54202.0,0.0'}}
budget['Facilities'][2][3] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,54201.0,0.0'}}
budget['Facilities'][2][4] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,54205.0,0.0'}}
budget['Facilities'][2][5] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,54206.0,0.0'}}
budget['Facilities'][2][6] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,54311.0,0.0',
                                       2: '10000000.0,*     ,313.0,10.0,2500.0,54311.0,0.0'}}
budget['Facilities'][2][7] = {'UCOA': {1: '10000000.0,*   ,321.0,10.0,2500.0,54312.0,0.0'}}
budget['Facilities'][2][8] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,54313.0,0.0'}}
budget['Facilities'][2][9] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,54321.0,0.0'}}
budget['Facilities'][2][10]= {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,54322.0,0.0'}}
budget['Facilities'][2][11]= {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,54323.0,0.0'}}
budget['Facilities'][2][12]= {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,54324.0,0.0'}}
budget['Facilities'][2][13]= {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,54402.0,0.0'}}
budget['Facilities'][2][14]= {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,54405.0,0.0'}}
budget['Facilities'][2][15]= {'UCOA': {1: '10000000.0,2000.0,321.0,10.0,2500.0,54601.0,0.0',
                                       2: '10000000.0,2900.0,321.0,10.0,2500.0,54601.0,0.0'}}
budget['Facilities'][2][16]= {'UCOA': {1: '10000000.0,*   ,321.0,10.0,2500.0,54602.0,0.0'}}
budget['Facilities'][2][17]= {'UCOA': {1: '10000000.0,*   ,321.0,10.0,2500.0,54901.0,0.0'}}
budget['Facilities'][2][18]= {'UCOA': {1: '10000000.0,*   ,313.0,10.0,2500.0,54902.0,0.0'}}
budget['Facilities'][2][18]= {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,54904.0,0.0'}}
#Other Purchased Services
budget['Facilities'][3][1] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,55201.0,0.0'}}
budget['Facilities'][3][2] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,55803.0,0.0'}}
#Other Purchased Services
budget['Facilities'][4][1] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,56112.0,0.0'}}
budget['Facilities'][4][2] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,56201.0,0.0'}}
budget['Facilities'][4][3] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,56201.0,0.0'}}
budget['Facilities'][4][4] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,56202.0,0.0'}}
budget['Facilities'][4][5] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,56203.0,0.0'}}
budget['Facilities'][4][6] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,56204.0,0.0'}}
budget['Facilities'][4][7] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,56211.0,0.0'}}
budget['Facilities'][4][8] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,56214.0,0.0'}}
budget['Facilities'][4][9] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,56215.0,0.0'}}
budget['Facilities'][4][10]= {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,56216.0,0.0'}}
budget['Facilities'][4][11]= {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,56217.0,0.0'}}
budget['Facilities'][4][12]= {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,56218.0,0.0'}}
budget['Facilities'][4][13]= {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,56219.0,0.0'}}
budget['Facilities'][4][14]= {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,56220.0,0.0'}}
#Property
budget['Facilities'][5][1] = {'UCOA': {1: '10000000.0,*     ,422.0,10.0,2500.0,57202.0,0.0'}}
budget['Facilities'][5][2] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,57305.0,0.0'}}
#Miscellaneous Expenses
budget['Facilities'][6][1] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,58101.0,0.0'}}
budget['Facilities'][6][2] = {'UCOA': {1: '10000000.0,2900.0,321.0,10.0,2500.0,58104.0,0.0'}}
budget['Facilities'][6][3] = {'UCOA': {1: '10000000.0,*     ,321.0,10.0,2500.0,58901.0,0.0'}}





In [ ]:
budget

In [ ]:


for budg in budget.keys():
    print(budg)
    for sect in budget[budg].keys():
        print('    ',budget_sections[sect])
        for i in budget[budg][sect].keys():
            rd = get_history(2014,budget[budg][sect][i])
            print(rd)
            rd = get_history(2015,budget[budg][sect][i])
            print(rd)
            rd = get_history(2016,budget[budg][sect][i])
            print(rd)
            rd = get_history(2017,budget[budg][sect][i])
            print(rd)
            rd = get_history(2018,budget[budg][sect][i])
            print(rd)
            print()

In [ ]:
def make_budget(fyear,start_yr):

    b = {'budgets':{}}
    
    b['years'] = np.arange(start_yr,fyear+1)
    
    for budg in budget.keys():                                                  #loop through budgets
        if (budg not in b['budgets'].keys()):                                              #add to dictionary if not there
            b['budgets'][budg] = {}
        for sect in budget[budg].keys():                                        #loop through sections
            if (sect not in b['budgets'][budg].keys()):                                    #add to dictionary if not there
                b['budgets'][budg][sect] = {}                               
            for itm in budget[budg][sect].keys():                               #loop through lines in section
                if (itm not in b['budgets'][budg][sect].keys()):                             #add to dictionary if not there
                    b['budgets'][budg][sect][itm] = {}
                    try:
                        b['budgets'][budg][sect][itm]['ucoa'] = budget[budg][sect][itm]['ucoa']
                    except KeyError:
                        b['budgets'][budg][sect][itm]['ucoa'] = {}
                for fy in np.arange(start_yr,fyear+1):                          #loop through fiscal years
                    b['budgets'][budg][sect][itm][fy] = get_history(fy,budget[budg][sect][itm])  #get history
    return(b)
                    
bd = make_budget(2018,2013)

bd

In [ ]:
title = 'Test budg'
fname = '../Test_budg.html'

file = open(fname, "w")

file.write("<html><body>\n" )
file.write("<h2>" + title + "</h2>\n")

grand_totals = np.zeros(len(bd['years']))
for budg in bd['budgets'].keys():
    file.write("<h3>" + budg + "</h3>\n")
    file.write('<table border="0" width="100%"><tr><td>')
    budget_totals = np.zeros(len(bd['years']))
    for bsect in bd['budgets'][budg].keys():
        if (len(bd['budgets'][budg][bsect].keys()) > 0):
            file.write('<tr style="background-color:Violet";"><td align="left"><b>' +\
                budget_sections[bsect] + '</b></td></tr>\n')
            file.write('<tr><td><table border="0" width="100%">\n')
            tstr = '<tr><td>&nbsp;</td><td>&nbsp;</td>'
            totals = np.zeros(len(bd['years']))
            for y in bd['years']:
                tstr = tstr + '<td align="center"><b>' + str(y) + '</b></td>'
            file.write(tstr + '</tr>\n')        

            for sect in bd['budgets'][budg][bsect].keys():
                ucoa = get_ucoa_codes(budget,budg,bsect,sect)
                tstr = '<tr><td width="18%">' + ucoa + '</td><td width="18%">' \
                    + bd['budgets'][budg][bsect][sect][y]['Desc'] + '</td>'
                ix=0
                for y in bd['years']: 
                    try:
                        tstr = tstr + '<td align="right">' +\
                            '${:,.2f}'.format(bd['budgets'][budg][bsect][sect][y]['Actual']) + '</td>'
                        totals[ix] += bd['budgets'][budg][bsect][sect][y]['Actual']
                        
                    except KeyError:
                        tstr = tstr + '<td align="center">-</td>'
                    ix+=1
                file.write(tstr + '</tr>\n')

            tstr = '<tr><td width="18%">&nbsp;</td><td width="18%">Total</td>'
            for ix in np.arange(len(bd['years'])):
                budget_totals[ix] += totals[ix]
                tstr = tstr + '<td align="right">' + '${:,.2f}'.format(totals[ix]) + '</td>'
            tstr += '</tr></table></td></tr>\n'
            file.write(tstr)
            
    file.write('<tr><td><table border="0" width="100%">\n')
    tstr = '<tr><td width="18%">&nbsp;</td><td width="18%"><b>Budget Total</b></td>'
    for ix in np.arange(len(bd['years'])):
        grand_totals[ix] += budget_totals[ix]
        tstr = tstr + '<td align="right"><b>' + '${:,.2f}'.format(budget_totals[ix]) + '</b></td>'
    tstr += '</tr></table></td></tr>\n'
    file.write(tstr)
    file.write('</td></tr></table>')
    
file.write('<tr><td><table border="0" width="100%">\n')
tstr = '<tr><td width="18%">&nbsp;</td><td width="18%"><b>Grand Total</b></td>'
for ix in np.arange(len(bd['years'])):
    tstr = tstr + '<td align="right"><b>' + '${:,.2f}'.format(grand_totals[ix]) + '</b></td>'
tstr += '</tr></table></td></tr>\n'
file.write(tstr)
file.write('</td></tr></table>')
    
file.write("</body></html>\n")

file.close()
